In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D,CuDNNLSTM
import pandas as pd
import time
data = pd.read_csv('train.csv')
y = pd.DataFrame(data['label'])
x = data.drop('label',axis=1)
x = x.values
x = np.array(x).reshape(-1,28,28)
x = np.array(x).reshape(-1,28,28,1)
y = y.values
y = np.array(y).reshape(-1)


In [0]:
x.shape


(42000, 28, 28, 1)

In [0]:
y.shape

(42000,)

In [0]:
def augment_data(dataset, dataset_labels, augementation_factor=1, use_random_rotation=True, use_random_shear=True, use_random_shift=True, use_random_zoom=True):
	augmented_image = []
	augmented_image_labels = []

	for num in range (0, dataset.shape[0]):

		for i in range(0, augementation_factor):
			# original image:
			augmented_image.append(dataset[num])
			augmented_image_labels.append(dataset_labels[num])

			if use_random_rotation:
				augmented_image.append(tf.contrib.keras.preprocessing.image.random_rotation(dataset[num], 20, row_axis=0, col_axis=1, channel_axis=1))
				augmented_image_labels.append(dataset_labels[num])

			if use_random_shear:
				augmented_image.append(tf.contrib.keras.preprocessing.image.random_shear(dataset[num], 0.2, row_axis=0, col_axis=1, channel_axis=2))
				augmented_image_labels.append(dataset_labels[num])

			if use_random_shift:
				augmented_image.append(tf.contrib.keras.preprocessing.image.random_shift(dataset[num], 0.2, 0.2, row_axis=0, col_axis=1, channel_axis=1))
				augmented_image_labels.append(dataset_labels[num])

	return np.array(augmented_image), np.array(augmented_image_labels)

In [0]:
x, y = augment_data(x,y)

In [0]:
x = np.array(x).reshape(-1,28,28)

In [0]:
y.shape

(168000,)

In [0]:
x = x/255
model = Sequential()
 
model.add(CuDNNLSTM(256, input_shape=(x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(256))
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

model.fit(x,
          y,
          epochs=15,validation_split=0.2)



Train on 134400 samples, validate on 33600 samples
Epoch 1/15
134400/134400 [==============================] - 158s 1ms/step - loss: 0.1932 - acc: 0.9399 - val_loss: 0.0611 - val_acc: 0.9830
Epoch 2/15
134400/134400 [==============================] - 127s 947us/step - loss: 0.0490 - acc: 0.9861 - val_loss: 0.0547 - val_acc: 0.9851
Epoch 3/15
134400/134400 [==============================] - 127s 945us/step - loss: 0.0320 - acc: 0.9906 - val_loss: 0.0606 - val_acc: 0.9860
Epoch 4/15
134400/134400 [==============================] - 127s 945us/step - loss: 0.0233 - acc: 0.9935 - val_loss: 0.0516 - val_acc: 0.9884
Epoch 5/15
134400/134400 [==============================] - 127s 946us/step - loss: 0.0168 - acc: 0.9953 - val_loss: 0.0787 - val_acc: 0.9838
Epoch 6/15
134400/134400 [==============================] - 127s 945us/step - loss: 0.0146 - acc: 0.9958 - val_loss: 0.0473 - val_acc: 0.9905
Epoch 7/15
134400/134400 [==============================] - 133s 988us/step - loss: 0.0126 - acc: 0

In [0]:
dest = pd.read_csv('test.csv')
test = dest.values
test = np.array(test).reshape(-1,28,28)
test = test/255
predictions = model.predict([test])
import numpy as np
result=[]
for item in predictions:
    result.append(np.argmax(item))
submit = pd.read_csv('sample_submission.csv')
submit['Label'] = result

In [0]:
submit.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [0]:
submit.to_csv("result_new.csv",index=False)

In [0]:
from google.colab import files
files.download('result_new.csv') 